In [3]:
import os
import numpy as np
import tifffile as tiff # for reading the original TIFF images
import cv2 # for saving JPEG images
import rasterio # for saving GeoTiff image
from tqdm.notebook import tqdm # for the beautiful progress-bars
from skimage.transform import rescale # for rescaling the images
from sklearn.impute import KNNImputer # for imputting the missing values

In [4]:
PATH_ORIGINAL_IMAGES = "../../../Nextcloud2/" # the folder with all the images
IDEAL_SIZE = 3000 # the ideal size of the output image (IDEAL_SIZE*IDEAL_SIZE pix), it is used for scaling RGB images
OUTPUT_MSP_NPY = "./data/drone_msp_numpy/" # output folder for MSP numpy matrixes
OUTPUT_MSP_FALSECOLOR = "./data/drone_false_color_msp/" # output folder for MSP falsecolor JPEG images
OUTPUT_RGB_JPG = "./data/drone_rgb_jpg/"
OUTPUT_RGB_GEOTIFF = "./data/drone_rgb/"

# Creating the ou tput directories if they do not exist
if not os.path.exists(OUTPUT_MSP_NPY):  
    os.mkdir(OUTPUT_MSP_NPY)
if not os.path.exists(OUTPUT_MSP_FALSECOLOR):  
    os.mkdir(OUTPUT_MSP_FALSECOLOR)
if not os.path.exists(OUTPUT_RGB_JPG):  
    os.mkdir(OUTPUT_RGB_JPG)
if not os.path.exists(OUTPUT_RGB_GEOTIFF):  
    os.mkdir(OUTPUT_RGB_GEOTIFF)
    
def image_minmax(im): # minmax normalizing image to the range from 0 to 1 for the JPEg output
    imax = im.max()
    imin = im.min()       
    im = (im - imin)/(imax - imin)
    return im

In [5]:
# listing all the pathes to the drone images
drfiles = []
for path, subdirs, files in os.walk(PATH_ORIGINAL_IMAGES):
    for name in files:
        drfiles = drfiles + [os.path.join(path, name)]


# listing all MSP images
nirfiles = [drfiles[i] for i in range(len(drfiles)) if any(x in drfiles[i] for x in ["_nir"])]
names_msp = [nirfiles[i].split("nir.tif")[0] for i in range(len(nirfiles))]

# listing all RGB images and selecting those for which MSP is not available
rgbfiles = [drfiles[i] for i in range(len(drfiles)) if ("_rgb.tif" in drfiles[i]) & ("_rgb.tif." not in drfiles[i])]
rgb_nams = [rg.split("\\")[-1].split("_RGB")[0] for rg in rgbfiles]
rgbfiles = np.array(rgbfiles)
rgb_wo_msp = rgbfiles[[np.all(np.char.find(names_msp, rn) < 0) for rn in rgb_nams]]
rgb_wo_msp = [rg.split("rgb.tif")[0] for rg in rgb_wo_msp]

In [4]:
# Reading each MSP image by band, filling in, stacking to one 4-band image (red, green, reg, nir),
# and saving it in .npy format
# Additionally, we save the falsecolor (NIR, red, green) image in JPEG format
abbrev = "msp"
bands = ["red","gre","reg","nir"]
imputer = KNNImputer(n_neighbors=9, weights="uniform") # the imputer for NA values
for i in tqdm(range(len(names_msp))): # for all the MSP images
    nam = names_msp[i].split("\\")[-1]
    print(i, "---- Image " + nam + " ----")
    nam = nam.split("_MSP")[0]
    
    full_im = []
    for band in bands: # reading MSP bands one by one
        path_im = names_msp[i] + band + ".tif"
        im = tiff.imread(path_im) # reading tiff image
        na_mask = im < -1000 # creating mask of NA values (they are represented as large negative values)
        print("Band: " + band + ", shape: " + str(im.shape) + 
              ", number of NA values:", len(im[na_mask]))

        # checking that there are not too much NA values
        if (len(im[na_mask])/(im.shape[0]*im.shape[1]*im.shape[0])) > 0.1: 
            print("Image", nam, ". Too many NA values to fill in. Proceeding without imputation.")
        else:
            im[na_mask] = np.nan
            im = imputer.fit_transform(im) # filling in NA values with the pre-defined imputer 
            full_im = full_im + [im] # adding current band to the list of bands

    full_im = np.stack(full_im, axis = -1) # stacking all bands to one image
    #Saving the MSP image as numpy matrix x*y*z, where x*y are image dimentions, and z=4 - number of bands
    np.save(OUTPUT_MSP_NPY + names_msp[i].split("\\")[-1] + "msp.npy", full_im)
    #Saving the MSP image as falsecolor JPEG (NIR, red, green) colorspace
    msp3 = full_im[:,:,[3,0,1]]
    msp3 = image_minmax(msp3)*255
    msp3 = msp3.astype(dtype=np.uint8)
    cv2.imwrite(OUTPUT_MSP_FALSECOLOR + names_msp[i].split("\\")[-1] + "falsecolor.jpg", \
                cv2.cvtColor(msp3, cv2.COLOR_RGB2BGR))

  0%|          | 0/2 [00:00<?, ?it/s]

25 ---- Image GL_ZAC_ZAC2_MSP_20190809_ ----
Band: red, shape: (3031, 3032), number of NA values: 6
Band: gre, shape: (3031, 3032), number of NA values: 1
Band: reg, shape: (3031, 3032), number of NA values: 0
Band: nir, shape: (3031, 3032), number of NA values: 0
26 ---- Image GL_ZAC_ZAC3_MSP_20190809_ ----
Band: red, shape: (3032, 3032), number of NA values: 161
Band: gre, shape: (3032, 3032), number of NA values: 902
Band: reg, shape: (3032, 3032), number of NA values: 420
Band: nir, shape: (3032, 3032), number of NA values: 201


In [10]:
# Reading each RGB image, filling it in and rescaling by band, stacking to one 3-band image (red, green, blue),
# and saving it in .tiff format with coordinates, and JPEG format

abbrev = "rgb"
imputer = KNNImputer(n_neighbors=9, weights="uniform") # the imputer for NA values

for i in tqdm(range(len(rgb_wo_msp))): #
    
    nam = rgb_wo_msp[i].split("\\")[-1]
    path_im = rgb_wo_msp[i] + "rgb.tif"
    im = tiff.imread(path_im) # reading tiff image
    
    na_mask = im < -1000 # creating mask of NA values (they are represented as large negative values)
    print(i, "---- Image " + nam + " ----")
    print("Shape:", im.shape, "Number of NA values:", len(im[na_mask]))

    # checking that there are not too much NA values
    if (len(im[na_mask])/(im.shape[0]*im.shape[1]*im.shape[2])) > 0.1: 
        print("Image", nam, ". Too many NA values to fill in.")
    else:
        im[na_mask] = np.nan
        full_im = [rescale(imputer.fit_transform(im[:,:,band]), 
                           IDEAL_SIZE/im.shape[0], anti_aliasing=False) 
                   for band in range(3)] # imputing image by band and rescaling it to IDEEAL_SIZE
        im = np.stack(full_im, axis=-1)
        
        # Saving the RGB image in GeoTiff format
        orig = rasterio.open(path_im) # reading out the original coordinates
        new_dataset = rasterio.open( 
            OUTPUT_RGB_GEOTIFF + nam + "rgb.tiff",
            'w',
            driver='GTiff',
            height=im.shape[0],
            width=im.shape[1],
            count=3,
            dtype=im.dtype,
            crs=orig.crs, # coordinates of original image
            transform=orig.transform,
        )
        new_dataset.write(np.transpose(im, [2,0,1])/255)
        new_dataset.close()

        # Saving the RGB image in JPEG format
        im = image_minmax(im)*255
        im = im.astype(dtype=np.uint8)
        cv2.imwrite(OUTPUT_RGB_JPG + nam + "rgb.jpg", \
                    cv2.cvtColor(im, cv2.COLOR_RGB2BGR))      
                
                

  0%|          | 0/1 [00:00<?, ?it/s]

0 ---- Image NO_SVB_SAST_RGB_20180801_ ----
Shape: (13112, 13112, 4) Number of NA values: 0
